# Análise dos dados
## Instalação de alguns pacotes

In [1]:
import pandas as pd
import sys
from glob import glob
from IPython.core.display import HTML
from sqlalchemy import create_engine

## Definição de algumas configurações

In [2]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 300)
display(HTML("<style>.container { width:100% !important; } div.prompt {min-width: 10ex;} .rendered_html table { font-size: 11px; }</style>"))

## Definição de algumas variáveis

In [3]:
path = '../data'
files = {'import': path + '/IMP_*.csv',
         'export': path + '/EXP_*.csv'}
ncm_file = path + '/NCM.csv'
uf_file = path + '/UF.csv'
db_file = path + '/trades.db'
kind = 'import'

## Inspeção dos dados brutos

Estas são as primeiras linhas do primeiro arquivo CSV do tipo (`kind`) de transação de mercadorias escolhido (`import` ou `export`):

In [4]:
%%timeit -n1 -r1
!head -n 4 {files[kind]}

==> ../data/IMP_2017.csv <==
"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"2017";"01";"54024400";"10";"399";"SP";"01";"0817800";4770;4770;47344
"2017";"01";"84834090";"11";"386";"SP";"01";"0817800";13055;48308;468136
"2017";"01";"84719014";"11";"399";"SP";"04";"0817700";3;6;146

==> ../data/IMP_2018.csv <==
"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"2018";"01";"85423190";"11";"160";"RS";"04";"0817700";139;1;261
"2018";"01";"85334011";"11";"399";"RS";"04";"1017801";1000;0;230
"2018";"02";"85049090";"10";"160";"PR";"04";"0917900";44;44;30066

==> ../data/IMP_2019.csv <==
"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"2019";"12";"85044021";"11";"249";"SP";"04";"0817700";53;44;8263
"2019";"12";"85183000";"11";"160";"GO";"04";"0817600";23734;1621;6987
"2019";"12";"39263000";"10";"023";"SP";"01";"0

## Criação de bancos de dados

In [5]:
# Disabled timing due to NameError. See https://github.com/ipython/ipython/issues/11659/
# %%timeit -n1 -r1
eng = create_engine(f'sqlite:///{db_file}')
conn = eng.connect()
chunk_size=500000

In [6]:
# %%timeit -n1 -r1
# for kind_of_trade, pattern in files.items():
#     for name in glob(pattern):
#         print(name)
#         for chunk in pd.read_csv(name, delimiter=';', chunksize=chunk_size):
#             chunk.to_sql(kind_of_trade, eng, if_exists='append')

In [7]:
# %%timeit -n1 -r1
# for chunk in pd.read_csv(ncm_file, delimiter=';', chunksize=chunk_size, encoding = 'ISO-8859-1'):
#     chunk.to_sql('ncm', eng, if_exists='append')

In [8]:
# %%timeit -n1 -r1
# for chunk in pd.read_csv(uf_file, delimiter=';', chunksize=chunk_size, encoding = 'ISO-8859-1'):
#     chunk.to_sql('uf', eng, if_exists='append')

## Tabelas disponíveis no banco da dados

In [9]:
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'", conn, index_col=["name"])

,type,tbl_name,rootpage,sql
name,,,,
import,table,import,2,"CREATE TABLE import (\n\t""index"" BIGINT, \n\t""CO_ANO"" BIGINT, \n\t""CO_MES"" BIGINT, \n\t""CO_NCM"" BIGINT, \n\t""CO_UNID"" BIGINT, \n\t""CO_PAIS"" BIGINT, \n\t""SG_UF_NCM"" TEXT, \n\t""CO_VIA"" BIGINT, \n\t""CO_URF"" BIGINT, \n\t""QT_ESTAT"" BIGINT, \n\t""KG_LIQUIDO"" BIGINT, \n\t""VL_FOB"" BIGINT\n)"
export,table,export,77444,"CREATE TABLE export (\n\t""index"" BIGINT, \n\t""CO_ANO"" BIGINT, \n\t""CO_MES"" BIGINT, \n\t""CO_NCM"" BIGINT, \n\t""CO_UNID"" BIGINT, \n\t""CO_PAIS"" BIGINT, \n\t""SG_UF_NCM"" TEXT, \n\t""CO_VIA"" BIGINT, \n\t""CO_URF"" BIGINT, \n\t""QT_ESTAT"" BIGINT, \n\t""KG_LIQUIDO"" BIGINT, \n\t""VL_FOB"" BIGINT\n)"
ncm,table,ncm,128139,"CREATE TABLE ncm (\n\t""index"" BIGINT, \n\t""CO_NCM"" BIGINT, \n\t""CO_UNID"" BIGINT, \n\t""CO_SH6"" BIGINT, \n\t""CO_PPE"" BIGINT, \n\t""CO_PPI"" BIGINT, \n\t""CO_FAT_AGREG"" BIGINT, \n\t""CO_CUCI_ITEM"" TEXT, \n\t""CO_CGCE_N3"" BIGINT, \n\t""CO_SIIT"" BIGINT, \n\t""CO_ISIC_CLASSE"" BIGINT, \n\t""CO_EXP_SUBSET"" FLOA..."
uf,table,uf,128849,"CREATE TABLE uf (\n\t""index"" BIGINT, \n\t""CO_UF"" BIGINT, \n\t""SG_UF"" TEXT, \n\t""NO_UF"" TEXT, \n\t""NO_REGIAO"" TEXT\n)"


## Número de transações

Contagem do número de linhas dos arquivos de transações de mercadorias. **Obs**: esse total inclui a primeira linha de cada arquivo (nomes das colunas)

In [10]:
%%timeit -n1 -r1
!wc -l {files[kind]}

  1823962 ../data/IMP_2017.csv
  1866512 ../data/IMP_2018.csv
  1904414 ../data/IMP_2019.csv
  5594888 total
1.19 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Aqui está o verdadeiro número de linhas na tabela recém criada:

In [11]:
%%timeit -n1 -r1
# conn.execute(f'SELECT COUNT(*) FROM {kind}').fetchall()
display(pd.read_sql(f'SELECT COUNT(*) FROM {kind}', conn))

,COUNT(*)
0,5594885


4.36 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Nomenclatura Comum do Mercosul (NCM)

In [12]:
%%timeit -n1 -r1
!wc -l {ncm_file}

13122 ../data/NCM.csv
135 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


O número de linhas na tabela recém criada é mostrado a seguir:

In [13]:
%%timeit -n1 -r1
# conn.execute('SELECT COUNT(*) FROM ncm').fetchall()
display(pd.read_sql('SELECT COUNT(*) FROM ncm', conn))

,COUNT(*)
0,13112


19.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


O comando `wc -l` retorna 10 linhas a mais pois, além de considerar a primeira linha, considera as 9 quebras de linha (`\n`) em células contendo strings:

In [14]:
%%timeit -n1 -r1
df = pd.read_sql('SELECT * FROM ncm', conn)
display(df[df.apply(lambda row: row.str.contains('\n').any(), axis=1)])

,index,CO_NCM,CO_UNID,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING
12666,12666,85235210,11,852352,3235,3235,3,89846,240,1000,2610,1106.0,\t\r\nCartões e etiquetas de acionamento por aproximação,Tarjetas y etiquetas de activación por proximidad,Proximity cards and proximity tags
12669,12669,27075010,10,270750,3215,3990,3,33525,240,2000,2011,8099.0,"Misturas que contenham trimetilbenzenos e etiltoluenos, como componentes majoritários","\r\nMezclas que contengan trimetilbenzenos y etiltoluenos, como componentes mayoritarios",\r\nMixtures containing trimethylbenzenes and ethyltoluenes as major constituents
12670,12670,54033110,10,540331,3275,3275,3,65174,240,2000,2030,1601.0,"Fio de raiom crus ou branqueados viscose, sem torção ou com torção não superior a 120 voltas por metro","Hiilado de rayón viscosa \r\r\ncrudos o blanqueados, sencillo, torsión<=120vueltas/m","Yarn of viscose rayon \r\r\ncrushed or bleached, single, twist<=120turns/meter"
12675,12675,95089043,10,950890,3070,3070,3,8946,110,2000,2829,8099.0,\t\r\n\r\nEquipamentos recreativos para parques aquáticos \t\r\n\r\nEquipamentos recreativos para parques aquáticos,Equipamiento recreativo para parques acuáticos.,Recreational equipment for water parks


3.09 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Unidades Federativas (UF)

In [15]:
%%timeit -n1 -r1
!wc -l {uf_file}

35 ../data/UF.csv
113 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
%%timeit -n1 -r1
# conn.execute('SELECT COUNT(*) FROM uf').fetchall()
display(pd.read_sql('SELECT COUNT(*) FROM uf', conn))

,COUNT(*)
0,34


18.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [17]:
display(pd.read_sql('SELECT * FROM uf LIMIT 5', conn))

,index,CO_UF,SG_UF,NO_UF,NO_REGIAO
0,0,11,RO,Rondônia,REGIAO NORTE
1,1,12,AC,Acre,REGIAO NORTE
2,2,13,AM,Amazonas,REGIAO NORTE
3,3,14,RR,Roraima,REGIAO NORTE
4,4,15,PA,Pará,REGIAO NORTE


## Informações gerais sobre a tabela de transações

In [18]:
# Disabled timing due to NameError. See https://github.com/ipython/ipython/issues/11659/
# %%timeit -n1 -r1
# 24.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
df = pd.read_sql(f'SELECT * FROM {kind}', conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5594885 entries, 0 to 5594884
Data columns (total 12 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   index       int64 
 1   CO_ANO      int64 
 2   CO_MES      int64 
 3   CO_NCM      int64 
 4   CO_UNID     int64 
 5   CO_PAIS     int64 
 6   SG_UF_NCM   object
 7   CO_VIA      int64 
 8   CO_URF      int64 
 9   QT_ESTAT    int64 
 10  KG_LIQUIDO  int64 
 11  VL_FOB      int64 
dtypes: int64(11), object(1)
memory usage: 512.2+ MB


## Quantidade de valores distintos por coluna

In [19]:
%%timeit -n1 -r1
display(df.nunique())

index         1904413
CO_ANO              3
CO_MES             12
CO_NCM           9166
CO_UNID            13
CO_PAIS           250
SG_UF_NCM          28
CO_VIA             10
CO_URF            132
QT_ESTAT       227247
KG_LIQUIDO     181475
VL_FOB         447867
dtype: int64

655 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Transações com maior valor FOB (Free on Board) em dólar (US$)

In [20]:
%%timeit -n1 -r1
# conn.execute(f'SELECT * FROM {kind} ORDER BY VL_FOB DESC LIMIT 5').fetchall()
display(pd.read_sql(f'SELECT * FROM {kind} ORDER BY VL_FOB DESC LIMIT 5', conn))

,index,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,661481,2018,11,89052000,11,105,RJ,10,917800,0,81451179,2215563252
1,1683231,2019,1,89052000,11,160,RJ,10,1017700,0,101748000,2085171726
2,1135488,2018,8,89052000,11,160,RJ,10,1017700,0,101748000,2071124563
3,252374,2018,2,89059000,11,105,RJ,10,1017700,2,90511000,1985998896
4,919422,2019,9,89052000,11,105,RJ,10,727600,2,76224380,1709212920


3.07 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Mercadorias com maior valor FOB total em dólar (US$)

Os maiores valores FOB totais, por mercadoria são os seguintes:

In [21]:
%%timeit -n1 -r1
query = f'''
SELECT CO_NCM,
       SUM(VL_FOB) total
FROM {kind}
GROUP BY CO_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,CO_NCM,total
0,27101921,18603831959
1,27090010,12664164160
2,27101241,9952530812


2.72 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Os nomes das mercadorias correspondentes aos códigos da coluna CO_NCM acima podem ser obtidos da tabela `ncm`:

In [22]:
%%timeit -n1 -r1
query = f'''
SELECT k.CO_NCM,
       NO_NCM_POR,
       SUM(VL_FOB) total
FROM {kind} k
JOIN ncm ON ncm.CO_NCM = k.CO_NCM
GROUP BY k.CO_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,CO_NCM,NO_NCM_POR,total
0,27101921,Gasóleo (óleo diesel),18603831959
1,27090010,Óleos brutos de petróleo,12664164160
2,27101241,Naftas para petroquimica,9952530812


5.83 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Estados com maior valor FOB total em dólar (US$)

In [23]:
%%timeit -n1 -r1
query = f'''
SELECT k.SG_UF_NCM, uf.NO_UF,
       SUM(VL_FOB) total
FROM {kind} k
JOIN uf ON uf.SG_UF = k.SG_UF_NCM
GROUP BY k.SG_UF_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,SG_UF_NCM,NO_UF,total
0,SP,São Paulo,175421880962
1,RJ,Rio de Janeiro,56304101371
2,SC,Santa Catarina,44973206316


3.63 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Valor FOB total anual em dólar (US$)

In [24]:
%%timeit -n1 -r1
query = f'''
SELECT CO_ANO,
       SUM(VL_FOB) total
FROM {kind}
GROUP BY CO_ANO
ORDER BY CO_ANO
'''
display(pd.read_sql(query, conn))

,CO_ANO,total
0,2017,150749494421
1,2018,181230568862
2,2019,177347934749


1.72 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Mercadorias com maior valor FOB total em dólar (US$), em São Paulo, em 2019

In [25]:
%%timeit -n1 -r1
year=2019
state='SP'
query = f'''
SELECT k.CO_ANO, k.SG_UF_NCM, n.NO_NCM_POR,
       SUM(VL_FOB) total
FROM {kind} k
JOIN ncm n ON k.CO_NCM=n.CO_NCM
JOIN uf u ON SG_UF_NCM=u.SG_UF
WHERE CO_ANO={year}
AND SG_UF_NCM="{state}"
GROUP BY k.CO_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,CO_ANO,SG_UF_NCM,NO_NCM_POR,total
0,2019,SP,Gasóleo (óleo diesel),1627159797
1,2019,SP,Outras partes para aparelhos de telefonia/telegrafia,1548336237
2,2019,SP,Óleos brutos de petróleo,1071774913


1.18 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
